In [1]:
import requests
import json
import pandas as pd
import folium
from pandas import json_normalize
from tabulate import tabulate
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import ipywidgets as widgets
from IPython.display import display
# API URL
url = "https://jathinmerda.github.io/map/NYC.json"

# Send a GET request to the API
response = requests.get(url)

response

<Response [200]>

In [2]:
bnb_data = response.json()
data = bnb_data.get("listing", [])
table_data = []
for listing in data:
    name = listing['name']
    host_name = listing['host_name']
    neighbourhood_group = listing['neighbourhood_group']
    neighbourhood=listing['neighbourhood']
    latitude=listing['latitude']
    longitude=listing['longitude']
    room_type = listing['room_type']
    price = listing['price']
    availability_365 = listing['availability_365']
    
    table_data.append([name, host_name,neighbourhood_group,neighbourhood,latitude,longitude, room_type, price, availability_365])

# Printing the table with headers
#print(tabulate(table_data, headers=["ID", "Name", "Host Name", "Neighbourhood Group","Neighbourhood","latitude","longitude", "Room Type", "Price", "Availability (in days)"], tablefmt="grid"))

In [3]:
import folium
from folium.plugins import MarkerCluster

listings = bnb_data.get("listing", [])

map_center = [40.730610, -73.935242]  # Adjust the map center according to your data
map_zoom = 10  # Adjust the initial zoom level
mymap = folium.Map(location=map_center, zoom_start=map_zoom, tiles="https://mt1.google.com/vt/lyrs=r&x={x}&y={y}&z={z}", attr="Google maps")

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(mymap)

# Iterate over each Airbnb listing
for listing in listings:
    name = listing['name']
    host_name = listing['host_name']
    neighbourhood_group = listing['neighbourhood_group']
    neighbourhood = listing['neighbourhood']
    latitude = listing['latitude']
    longitude = listing['longitude']
    room_type = listing['room_type']
    price = listing['price']
    availability_365 = listing['availability_365']
    
    # Create popup text for the marker
    popup_text = f"{name}<br>Host: {host_name}<br>Neighbourhood: {neighbourhood}<br>Room Type: {room_type}<br>Price: ${price}<br>Availability (in days): {availability_365}"
    
    # Create a marker and add it to the MarkerCluster
    folium.Marker(
        location=[latitude, longitude], 
        popup=popup_text,
        tooltip=name,
        icon=folium.Icon(icon='home', prefix='fa', color='blue')
    ).add_to(marker_cluster)  # Add marker to the MarkerCluster

# Display the map
mymap


In [4]:
import pandas as pd
import folium
import panel as pn
import requests
from folium.plugins import MarkerCluster

# Activate the Panel extension for the notebook
pn.extension()

# Function to fetch Airbnb listing data from API
def fetch_airbnb_data(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()['listing']
    else:
        print(f"Failed to fetch data from API: {response.text}")
        return []

# Function to create a map centered on the selected neighborhood with marker clustering
def create_folium_map(data, zoom):
    # Initialize a Folium map centered on the first listing's location
    if len(data) > 0:
        center_lat = data[0]['latitude']
        center_lng = data[0]['longitude']
    else:
        center_lat, center_lng = 40.73061, -73.935242  # Default center for NYC
    
    mymap = folium.Map(location=[center_lat, center_lng], zoom_start=zoom)

    # Create a MarkerCluster layer
    marker_cluster = MarkerCluster().add_to(mymap)

    # Add markers for each listing in the DataFrame to the MarkerCluster layer
    for listing in data:
        latitude = listing['latitude']
        longitude = listing['longitude']
        name = listing['name']
        neighborhood = listing['neighbourhood']
        room_type = listing['room_type']
        price = listing['price']
        popup_text = f"{name} - {neighborhood} - Room Type: {room_type} - Price: ${price} per night"

        # Create a custom marker with a FontAwesome 'home' icon
        icon = folium.Icon(icon='home', prefix='fa', color='blue')
        marker = folium.Marker(
            location=[latitude, longitude], 
            popup=popup_text,
            tooltip=name,
            icon=icon
        )
        
        # Add the marker to the MarkerCluster layer
        marker.add_to(marker_cluster)

    # Return the map
    return mymap

# Function to update the map based on the selected filters
def update_map(neighbourhood, room_type, max_price, min_nights, width, height):
    # Fetch Airbnb data from API
    api_url = 'https://jathinmerda.github.io/map/NYC.json'
    airbnb_data = fetch_airbnb_data(api_url)

    # Filter the data based on selected filters
    filtered_data = [listing for listing in airbnb_data if
                     (neighbourhood == 'All' or listing['neighbourhood_group'] == neighbourhood) and
                     (room_type == 'All' or listing['room_type'] == room_type) and
                     listing['price'] <= max_price and
                     listing['minimum_nights'] >= min_nights]

    # Create Folium map with the filtered data and marker clustering
    folium_map = create_folium_map(filtered_data, zoom=10)  # Adjust zoom level as needed
    
    # Panel doesn't directly render Folium maps, so we need to render it as HTML
    return pn.pane.HTML(folium_map._repr_html_(), width=width, height=height)

# Define the neighborhood group selector widget
neighbourhood_groups = ['All', 'Brooklyn', 'Manhattan', 'Queens', 'Bronx', 'Staten Island']  # Update with your neighborhoods
neighbourhood_selector = pn.widgets.Select(name='Neighbourhood', options=neighbourhood_groups)

# Define the room type selector widget
room_types = ['All', 'Private room', 'Entire home/apt', 'Shared room']  # Update with your room types
room_type_selector = pn.widgets.Select(name='Room Type', options=room_types)

# Define price and minimum nights sliders
price_slider = pn.widgets.IntSlider(name='Max Price ($)', start=10, end=500, step=10, value=500)
nights_slider = pn.widgets.IntSlider(name='Min Nights', start=1, end=30, step=1, value=1)

# Bind the update_map function to the widget callbacks
width = 1000
height = 700
map_pane = pn.bind(update_map, neighbourhood_selector, room_type_selector, price_slider, nights_slider, width, height)

# Layout the dashboard
dashboard = pn.Column(
    '## Interactive Airbnb Listings Map',
    pn.Row(
        neighbourhood_selector,
        room_type_selector,
        price_slider,
        nights_slider,
        align='center'
    ),
    map_pane
)

# Serve the dashboard
dashboard.servable()


Column
    [0] Markdown(str)
    [1] Row(align='center')
        [0] Select(name='Neighbourhood', options=['All', 'Brooklyn', ...], value='All')
        [1] Select(name='Room Type', options=['All', 'Private room', ...], value='All')
        [2] IntSlider(end=500, name='Max Price ($)', start=10, step=10, value=500)
        [3] IntSlider(end=30, name='Min Nights', start=1, value=1)
    [2] ParamFunction(function, _pane=HTML, defer_load=False)